In [1]:
import numpy as np
import pandas as pd

In [3]:
# load datasets
datasets = {}
years = range(2016, 2024)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='utf-8', dtype = str)
    except:
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='Windows 1251', dtype = str)
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...
success
EIE 2019 Loading...
success
EIE 2020 Loading...
success
EIE 2021 Loading...
success
EIE 2022 Loading...
success
EIE 2023 Loading...
success


In [27]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year

In [28]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if col.startswith('eo')])

['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']


In [32]:
organizations = pd.DataFrame(columns=['eoparent', 'year'])

In [34]:
for year, dataset in datasets.items():
    organizations = pd.concat([organizations, pd.DataFrame(dataset.loc[:, ['eoparent', 'year']])], ignore_index=True)

organizations

,eoparent,year
0,Відділ освіти Мелітопольської районної державн...,2016
1,"Управління освіти, молоді та спорту Красилівсь...",2016
2,NaN,2016
3,Управління освіти Чернівецької міської ради,2016
4,NaN,2016
...,...,...
2490048,Управління освіти Великоновосілківської селищн...,2023
2490049,NaN,2023
2490050,Відділ освіти Сихівського та Личаківського рай...,2023
2490051,Міністерство освіти і науки України,2023


In [35]:
organizations.drop_duplicates(inplace=True)
# When the paricipant graduated in the previous years 
# the corresponding information about school (everything with 'eo') is NaN.
# We drop these rows.
organizations.dropna(subset=organizations.columns.difference(['year']), 
                      how = 'all', inplace=True)
organizations.reset_index(drop=True, inplace=True)
organizations

,eoparent,year
0,Відділ освіти Мелітопольської районної державн...,2016
1,"Управління освіти, молоді та спорту Красилівсь...",2016
2,Управління освіти Чернівецької міської ради,2016
3,Відділ освіти Дружківської міської ради,2016
4,Управління освіти і науки Тернопільської міськ...,2016
...,...,...
9907,"Відділ освіти, молоді та спорту Плисківської с...",2023
9908,Відділ освіти та культури Нововасилівської сел...,2023
9909,Міністерство оборони України,2023
9910,"Відділ освіти, культури, молоді, спорту, туриз...",2023


In [36]:
# check whether there are duplicates by name and region
organizations[organizations.duplicated(subset=['eoparent', 'year'], keep=False)]

,eoparent,year


In [37]:
organizations_aggr = organizations.groupby(['eoparent'])['year'].agg(lambda x: list(x.unique())).reset_index()
organizations_aggr

,eoparent,year
0,Відділ освіти Міловської районної державної а...,[2016]
1,"Відділ освіти, молоді та спорту Маловисківськ...",[2017]
2,"Відділ освіти, молоді та спорту Ямпільської р...",[2016]
3,"Відділ освіти, молоді та спорту Ямпільської р...","[2017, 2018]"
4,Новотроїцький відділ освіти районної державно...,[2016]
...,...,...
4370,Щиборівська сільська рада Хмельницького району...,[2023]
4371,Яблунівська сільська рада,"[2022, 2023]"
4372,відділ освіти Покровської РДА Донецької област...,[2020]
4373,відділ освіти Судововишнянської об'єднаної тер...,"[2018, 2019]"


In [38]:
def create_id(dataset, attr, id):
    dataset.loc[:,id] = dataset[attr]

    abbr = {'B':'В', 'E':'Е', 'H':'Н', 'M':'М',
            'T':'Т', 'I':'І', 'O':'О', 'P':'Р',
            'A':'А', 'K':'К', 'X':'Х', 'C':'С',
            'c':'с', 'e':'е', 'i':'і', 'o':'о',
            'y':'у', 'p':'р', 'a':'а', 'k':'к',
            'x': 'х', 'ы':'и',
            ' РАДИ':' ', ' РАДА': ' ',
            ' ради':' ', ' рада': ' ',
            'КЗ ':'Комунальний заклад ', 'КУ ':'Комунальна установа ',
            'ОНЗ ':'Опорний навчальний заклад ', 'ДНЗ ':'Державний навчальний заклад ',
            'ОЗЗСО ':'Опорний заклад загальної середньої освіти ', 'ЗНЗ ':'Загальноосвітній навчальний заклад ',
            'ЗЗСО ':'Заклад загальної середньої освіти ', 'ПЗО ':'Приватний заклад освіти ', 
            'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
            'ЗОШ ':'Загальноосвітня школа ', 'НВК ':'Навчально-виховний комплекс ', 
            'ВСП ': 'Відокремлений структурний підрозділ', 'СЗШ ':'Середня загальноосвітня школа ', 
            'ТОВ ': 'Товариство з обмеженою відповідальністю ', 
            'ОЗ ': 'Опорний заклад ', "-":'',  "–":'', "-":'', "ЗШ ": "Загальноосвітня школа ",
            "'":'', "’":'', ",":'',"„":"","“": "", "«":'', "»":'',  "*":"", '"':"", ":":"", 
            ";": "", "!": "", "?":"", "_x000d_":'', '_x000D_':'', "_":'', 'нн':'н', 'НН':'Н', "№":""}

    for ab in abbr:
        dataset.loc[:,id] = dataset[id].str.replace(ab, abbr[ab])

    dataset.loc[:,id] = dataset[id].str.replace(r"[\(\[\{].*?[\)\]\}]", '', regex=True)

    brackets = {"(":'', ")":'', 
            "[":'', "]":'', "}":'', "{":''}
    
    for bra in brackets:
        dataset.loc[:,id] = dataset[id].str.replace(bra, brackets[bra])

    def del_rayon(name):
        t = name.split()
        t_lower = name.lower().split()
        for s in ['району', 'район']:
            if s in t_lower:
                i = t_lower.index(s)
                del t[i-1:i+1]
                return ' '.join(t)
        return name
    
    dataset['id'] = dataset['id'].apply(del_rayon)

    # def del_gromada(name):
    #     t = name.split('територіальної громади')
    #     t_lower = name.lower().split()
    #     if 'обєднаної' in t_lower:
    #         t_lower[0] = t_lower[0].split('обєднаної')[0]
    #         if 'рад' not in t_lower[0]:
    #             t_lower[0] = t_lower[0].split().pop()[0]
    #     else:
    #         if 'рад' not in t_lower[0]:
    #             t_lower[0] = t_lower[0].split()[0]
    #     return name

    # dataset['id'] = dataset['id'].apply(del_gromada)
    
    dataset.loc[:,id] = dataset[id].str.replace(r'[\s\r\n\t]+', '', regex=True)

    # dataset.loc[dataset[id].str.endswith('обл'), id] = dataset.loc[dataset['id'].str.endswith('обл'), id] + '.'
    # dataset.loc[:,id] = dataset[id].str.replace('IIII', '3')
    # dataset.loc[:,id] = dataset[id].str.replace('III', '2')
    dataset.loc[:,id] = dataset[id].str.replace('ІШ', '3')
    dataset.loc[:,id] = dataset[id].str.replace('ІІІІ', '3')
    dataset.loc[:,id] = dataset[id].str.replace('ІІІ', '2')
    dataset.loc[:,id] = dataset[id].str.replace('||||', '3')
    dataset.loc[:,id] = dataset[id].str.replace('|||', '2')
    dataset.loc[:,id] = dataset[id].str.lower()

    dataset['id'] = dataset['id'].apply(lambda s: s if (len(s) and s[-1].isdigit()) else s[:-1])


    dct = {"no":"",
           'ої':'а',
           'села':'с', 'село':'с', 'селещна':'селищна',
           'сільська':'', 'сільськ':'', 'сільска':'', 'сільск':'',
           'района':'', 'район':'',
           'селищна':'', 'селищн':'',
           'міська':'', 'міськ':'',
           'обєднана':'',
           'ст.':'ступ', "ступ.":"ступ", 
           "ступеня":"ступ", "ступенів":"ступ", 
           "ступеню":"ступ",
           "області":"обл", "область":"обл", "област":"обл", 
           "місто":"м", "міста":"м",
           'і':'', 'ї':'', 
           'ґ':'г', 'є':'',
            ".":'',
           'загальносвтня':'загальноосвтня',
           'сльско':'сльсько',
           "фаховий":"",
           "комунальнийзакладосвти":"",
           "комунальногозакладу":'',
           "комунальнийзаклад":"",
           "комунальнийопорнийзакладосвти":'',
           "комунальнийопорнийзаклад":'',
           'державнаорганзацяустановазакладдержавнийнавчальнийзаклад':'',
           'державнаорганзацяустановазаклад':'',
           'державнийнавчальнийзаклад':'',
           'державнийзаклад':'',
           'ншорганзацйноправовформи':'',
           'товариствозобмеженоювдповдальнстю':'',
           'вдокремленийструктурнийпдроздл':'',
           'комунальнаорганзаця':'',
           'комунальнаустанова':'',
           'обднанатериторальнагромади':'',
           'територальнагромади':''}

    for symb in dct:
        dataset.loc[:,id] = dataset[id].str.replace(symb, dct[symb])
    

    dataset.loc[:,id] = dataset[id].str.replace('"', '', regex = True)
    dataset.loc[:,id] = dataset[id].str.replace(r'\\', '')

    return dataset


In [40]:
create_id(organizations_aggr, 'eoparent', 'id')

,eoparent,year,id
0,Відділ освіти Міловської районної державної а...,[2016],вддлосвтимловськадержавнаадмнстрацлуганськаобл
1,"Відділ освіти, молоді та спорту Маловисківськ...",[2017],вддлосвтимолодтаспортумаловисквськао
2,"Відділ освіти, молоді та спорту Ямпільської р...",[2016],вддлосвтимолодтаспортуямпльськадержавнаадмнстрац
3,"Відділ освіти, молоді та спорту Ямпільської р...","[2017, 2018]",вддлосвтимолодтаспортуямпльськадержавнаадмнстрац
4,Новотроїцький відділ освіти районної державно...,[2016],новотрацькийвддлосвтидержавнаадмнстрац
...,...,...,...
4370,Щиборівська сільська рада Хмельницького району...,[2023],щиборвськахмельницькаобл
4371,Яблунівська сільська рада,"[2022, 2023]",яблунвська
4372,відділ освіти Покровської РДА Донецької област...,[2020],вддлосвтипокровськардадонецькаобл
4373,відділ освіти Судововишнянської об'єднаної тер...,"[2018, 2019]",вддлосвтисудововишнянськатериторальнагромад


# MON data 2023

## Full name

In [41]:
#load data
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [42]:
# create new with id
schools_data_id = schools_data.copy()
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...


In [43]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [44]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
3906,Брониківська загальноосвітня школа I-II ступен...,145546,так,NaN,6669916,працює,школа,Комунальна,1824080401,Житомирська область,...,NaN,bronykyzosh@ukr.net,http://bronuku.at.ua/,О.М. Алєксєєва,ні,так,ні,ні,1,брониквськазагальноосвтняшкола2ступжитомирськаобл
3940,Вепринський навчально-виховний комплекс І ступеню,146135,так,NaN,2206723,працює,навчально-виховний комплекс (об'єднання),Комунальна,1825081101,Житомирська область,...,NaN,veprinnvk@ukr.net,NaN,Г.С. Миненко,ні,так,ні,ні,1,вепринськийнавчальновиховнийкомплексступен
4077,Калинівська загальноосвітня школа I-II ступені...,146872,так,NaN,6669980,працює,школа,Комунальна,1824084802,Житомирська область,...,NaN,kalinivka@i.ua,http://kalin-shkola.at.ua/,І.Й. Жигун,ні,так,ні,ні,1,калинвськазагальноосвтняшкола2ступжитомирськаобл
4118,Міжнародний приватний навчально-виховний компл...,149486,так,Міжнародний приватний НВК імені Януша Корчака,4068919,працює,навчально-виховний комплекс (об'єднання),Комунальна,1822510100,Житомирська область,...,NaN,grigory.khmelnytsky@gmail.com,NaN,NaN,ні,ні,ні,ні,1,мжнароднийприватнийнавчальновиховнийкомплексме...
4354,Селецька загальноосвітня школа I-II ступенів Н...,148312,так,NaN,6670486,працює,школа,Комунальна,1823787401,Житомирська область,...,NaN,school_selets@i.ua,NaN,І.В. Ніконенко,ні,так,ні,ні,1,селецьказагальноосвтняшкола2ступжитомирськаобл
15978,Зюбриська загальноосвітня школа І ступеня Мона...,140731,так,NaN,2363037,працює,школа,Комунальна,7123481701,Черкаська область,...,NaN,Zybr.chkola@gmail.com,NaN,NaN,ні,так,ні,ні,1,зюбриськазагальноосвтняшколаступмонастирищенсь...


In [45]:
schools_data_id['EDRPOU']=schools_data_id['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [46]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU.str.len() == 8]

In [47]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,NaN,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл


In [48]:
schools_data_id = schools_data_id.drop_duplicates(subset=['id', 'EDRPOU'], keep='last')

In [49]:
schools_data_id[schools_data_id.duplicated(subset=['id'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
91,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС А""",151623,так,"ПЗО ""УОЛД СКУЛ КАМПУС А""",42180923,працює,школа,Приватна,8036300000,КИЇВ,...,NaN,campus.a@worldschool.com.ua,NaN,Директор Бадаєва Інна Миколаївна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус
254,Технічний ліцей м. Києва,145237,так,Технічний ліцей м. Києва,22876023,працює,ліцей,Комунальна,8036600000,КИЇВ,...,NaN,dtl-licej@ukr.ua,http://tl.kyiv.ua/,Директор Андрієнко Анатолій Михайлович,ні,ні,ні,ні,NaN,технчнийлцеймкив
268,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС Б""",176647,так,"ПЗО ""УОЛД СКУЛ КАМПУС Б""",43594721,працює,школа,Приватна,8000000000,КИЇВ,...,NaN,NaN,NaN,Директор Давидова Наталія Леонідівна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус
584,Технічний ліцей Шевченківського району м. Києва,149078,так,Технічний ліцей,22881946,працює,ліцей,Комунальна,8039100000,КИЇВ,...,(044)501-16-36,texlyceum@ukr.net,techlyceum.kiev.ua,Директор Ілюшина Олена Олександрівна,ні,ні,ні,ні,844,технчнийлцеймкив
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,NaN,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвськийзакладзагальнасередньаосвти3ступхус...
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,NaN,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвськийзакладзагальнасередньаосвти3ступхус...
6818,Гоголівський ліцей Великодимерської селищної р...,146689,так,Гоголівський ліцей,25299425,працює,ліцей,Комунальна,3221281601,Київська область,...,(04594)27395,gogoliv.school@gmail.com,http://gogolivska-zosh.kiev.sch.in.ua/,Директор Дяченко Оксана Іванівна,Опорний заклад,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл
6819,Гоголівський ліцей Великодимерської селищної р...,147130,так,філія Гоголівського ліцею,25299342,працює,ліцей,Комунальна,3221281601,Київська область,...,NaN,gogoliv-school2@ukr.net,http://gogoliv-school2.kiev.sch.in.ua,Завідувач філією Мишкоріз Любов Миколаївна,Філія,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл
11534,"Комунальний заклад ""Розсошенська гімназія Щерб...",139999,так,Розсошенська гімназія,21071866,працює,гімназія,Комунальна,5324087705,Полтавська область,...,NaN,rzsgymn@gmail.com,http://rzsschool.org.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,834,розсошенськагмназящербанвськаполтавськаобл
11535,"Комунальний заклад ""Розсошенська гімназія Щерб...",151527,так,Розсошенська гімназія,52852882,ліквідовано,гімназія,Комунальна,5324087705,Полтавська область,...,NaN,rzsgymn@gmail.com,http://rzsschool.ucoz.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,NaN,розсошенськагмназящербанвськаполтавськаобл


In [50]:
schools_data_id = schools_data_id[~schools_data_id.EDRPOU.isin(['34440724', '52852882', '58826264', 
                                                                '33329415', '26271178', '27646632', 
                                                                '04393999', '24623237', '25649174',
                                                                '25649172'])]

In [51]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id'], keep=False)].index.values.tolist(), inplace=True)

In [52]:
organizations_aggr_ed = organizations_aggr.merge(schools_data_id[['EDRPOU', 'id']], on=['id'], how='left')
organizations_aggr_ed.head()

,eoparent,year,id,EDRPOU
0,Відділ освіти Міловської районної державної а...,[2016],вддлосвтимловськадержавнаадмнстрацлуганськаобл,NaN
1,"Відділ освіти, молоді та спорту Маловисківськ...",[2017],вддлосвтимолодтаспортумаловисквськао,NaN
2,"Відділ освіти, молоді та спорту Ямпільської р...",[2016],вддлосвтимолодтаспортуямпльськадержавнаадмнстрац,NaN
3,"Відділ освіти, молоді та спорту Ямпільської р...","[2017, 2018]",вддлосвтимолодтаспортуямпльськадержавнаадмнстрац,NaN
4,Новотроїцький відділ освіти районної державно...,[2016],новотрацькийвддлосвтидержавнаадмнстрац,NaN


In [53]:
organizations_aggr_ed[organizations_aggr_ed['EDRPOU'].isna()].year.value_counts()

year
[2023]                                  575
[2022, 2023]                            423
[2022]                                  396
[2021]                                  281
[2016]                                  236
                                       ... 
[2018, 2019, 2021]                        1
[2020, 2022]                              1
[2017, 2019, 2020, 2022, 2023]            1
[2018, 2020]                              1
[2017, 2018, 2019, 2020, 2022, 2023]      1
Name: count, Length: 102, dtype: int64

In [54]:
organizations_aggr_ed.loc[organizations_aggr_ed['EDRPOU'].isna(), 'EDRPOU'] = ''

In [55]:
organizations_aggr_ed[organizations_aggr_ed.EDRPOU != '']

,eoparent,year,id,EDRPOU
3562,Київський університет імені Бориса Грінченка,[2023],кивськийунверситетменборисагрнченк,02136554
3706,Седнівський ліцей Седнівської селищної ради Че...,"[2022, 2023]",седнвськийлцейседнвськачернгвськаобл,26543755


# Youcontrol

In [56]:
youcontrol =  pd.read_excel('./school_data/Youcontrol_historical_name.xlsx', dtype='str')
youcontrol 

,Код ЄДРПОУ,Назва,Коротка назва
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN
...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [57]:
youcontrol.columns

Index(['Код ЄДРПОУ', 'Назва', 'Коротка назва'], dtype='object')

## long name

In [58]:
youcontrol[youcontrol.duplicated(subset=['Назва', 'Код ЄДРПОУ'], keep=False)]

,Код ЄДРПОУ,Назва,Коротка назва
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26
119,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
121,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
144,00854423,"КООПЕРАТИВ ""НОВОКРОПИВНИЦЬКИЙ""",NaN
...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80919,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [59]:
youcontrol_long = youcontrol.drop_duplicates(subset=['Назва', 'Код ЄДРПОУ'], keep='first')

In [60]:
create_id(youcontrol_long, 'Назва', 'id')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_35862/2993025049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id] = dataset[attr]
/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_35862/2993025049.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(del_rayon)
/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_35862/2993025049.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,Код ЄДРПОУ,Назва,Коротка назва,id
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN,вдкритеакцонернетовариствопвдентеплоенергомонта
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",публчнеакцонернетовариствопвдентеплоенергомонта
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",приватнеакцонернетовариствопвдентеплоенергомонта
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
...,...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [61]:
youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_35862/2371828141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [62]:
youcontrol_long[(youcontrol_long.EDRPOU.str.len() < 8)&(youcontrol_long.EDRPOU.str.len().notna())]

,EDRPOU,Назва,Коротка назва,id


In [63]:
youcontrol_long[youcontrol_long.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
48,00192471,"КОЛЕКТИВНЕ СПЕЦІАЛІЗОВАНЕ ПІДПРИЄМСТВО ""УКРЕНЕ...",NaN,колективнеспецалзованепдпримствоукренергочормет2
49,00192471,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ СП...,NaN,колективнеспецалзованепдпримствоукренергочормет2
67,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВООЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
...,...,...,...,...
80917,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ,деребчинськасереднязагальноосвтняшкола3ступвни...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальнасередньаосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальнасередн...


In [64]:
youcontrol_long = youcontrol_long.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [65]:
youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мжшкльнийнавчальновиробничийкомбна
494,02218789,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) МУЗИ...",NaN,музичнашкола2
498,02218884,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола3
503,02218921,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола2
719,02542129,МИКОЛАЇВСЬКИЙ ПРОФЕСІЙНИЙ ЛІЦЕЙ,NaN,миколавськийпрофесйнийлце
...,...,...,...,...
80889,45117367,"ФЕРМЕРСЬКЕ ГОСПОДАРСТВО ""ДОВІРА Б""","ФГ ""ДОВІРА Б""",фермерськегосподарстводовра
80903,45251511,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашколаздошкльнимпдроздломс...


In [66]:
youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_35862/2842222147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [67]:
def merging(df1, df2, lst_on):
    output = df1.merge(df2, on=lst_on, how='left')
    output.loc[output['EDRPOU_y'].isna(), 'EDRPOU_y'] = ''
    output.loc[output['EDRPOU_x'] == '', 'EDRPOU_x'] = output.loc[output['EDRPOU_x'] == '', 'EDRPOU_y']
    output = output.drop(columns=['EDRPOU_y'], axis=1)
    output = output.rename(columns = {'EDRPOU_x':'EDRPOU'})
    print('Percentage matched:',round(output[output['EDRPOU'] != ''].shape[0]/output.shape[0]*100,1), '%')
    return output

In [70]:
organizations_aggr_long = merging(organizations_aggr_ed, youcontrol_long[['id', 'EDRPOU']], ['id'])
organizations_aggr_long.head(30)

Percentage matched: 0.3 %


,eoparent,year,id,EDRPOU
0,Відділ освіти Міловської районної державної а...,[2016],вддлосвтимловськадержавнаадмнстрацлуганськаобл,
1,"Відділ освіти, молоді та спорту Маловисківськ...",[2017],вддлосвтимолодтаспортумаловисквськао,
2,"Відділ освіти, молоді та спорту Ямпільської р...",[2016],вддлосвтимолодтаспортуямпльськадержавнаадмнстрац,
3,"Відділ освіти, молоді та спорту Ямпільської р...","[2017, 2018]",вддлосвтимолодтаспортуямпльськадержавнаадмнстрац,
4,Новотроїцький відділ освіти районної державно...,[2016],новотрацькийвддлосвтидержавнаадмнстрац,
5,Іванівська сільська рада,[2023],ванвська,
6,Інформаційно-сервісний методичний центр відділ...,[2019],нформацйносервснийметодичнийцентрвддлуосвтимол...,
7,Інше (для приватних закладів),"[2018, 2019, 2020, 2021, 2022, 2023]",нш,
8,Іршанська селищна територіальна громада Хороші...,[2017],ршанськатериторальнагромадажитомирськаобл,
9,Ананьївський відділ освіти районної державної ...,"[2016, 2017, 2018, 2019, 2020]",ананьвськийвддлосвтидержавнаадмнстрац,


In [2]:
#!pip install -U duckpy
from duckpy import Client
from tqdm import tqdm
import time
tqdm.pandas()

def search_opendatabot(school_name, max_retry=20):
    """
    Search for school in opendatabot.ua
    """
    client = Client()
    opendatabot_url = ''
    retry = 0
    while not opendatabot_url and retry < max_retry:
        try:
            results = client.search(school_name, limit=20)
        except:
            time.sleep(20)
            continue
        for site in results:
            if site['url'].startswith('https://opendatabot.ua/c/'):
                opendatabot_url = site['url']
                break
        retry += 1
    return {'eoname': school_name, "url": opendatabot_url}

search_opendatabot("Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради")

{'eoname': 'Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради',
 'url': 'https://opendatabot.ua/c/26229527'}

In [10]:
opendatabot_links_new = []
for eoname in tqdm(organizations['eoparent']):
    opendatabot_links_new.append(search_opendatabot(eoname))

100%|██████████| 3799/3799 [3:24:08<00:00,  3.22s/it]    


In [12]:
opendatabot_df = pd.DataFrame(opendatabot_links_new)

In [13]:
opendatabot_df

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",
3795,"Відділ освіти, культури, молоді та спорту Біло...",
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",
3797,Гуманітарний відділ Литовезької сільської ради...,


In [14]:
opendatabot_df[opendatabot_df.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3781,"Відділ освіти, культури, молоді та спорту Нижн...",https://opendatabot.ua/c/44041491
3782,"Відділ освіти, культури, спорту та туризму Рай...",https://opendatabot.ua/c/43915446
3783,Гуманітарний відділ виконавчого комітету Узинс...,https://opendatabot.ua/c/41973438
3784,Шабівська сільська рада,https://opendatabot.ua/c/04378043


In [15]:
opendatabot_df.to_csv('organizations_link.csv', index=False)

# Next try

In [129]:
unmatched_link = pd.read_csv('organizations_link.csv', dtype=str)
unmatched_link

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343


In [130]:
unmatched_new = unmatched_link[~unmatched_link.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]
unmatched_new

,eoname,url
62,Відділ освіти Броварської райдержадміністрації,NaN
63,"Управління освіти, молоді та спорту Дубровицьк...",NaN
74,Відділ освіти виконкому Дзержинської районної ...,NaN
85,Відділ освіти Городнянської районної державної...,NaN
86,Управління освіти і науки Чернігівської обласн...,NaN
...,...,...
3663,"Управління освіти, культури, туризму, молоді т...",NaN
3664,Відділ освіти Гребінківської селищної ради,NaN
3665,Відділ освіти Зміївської міської ради Чугуївсь...,NaN
3666,"Відділ освіти, культури, молоді та спорту Мар'...",NaN


In [131]:
opendatabot_links_new = []
for eoname in tqdm(unmatched_new['eoname']):
    opendatabot_links_new.append(search_opendatabot(eoname))

100%|██████████| 173/173 [12:26<00:00,  4.31s/it]


In [132]:
opendatabot_df_new = pd.DataFrame(opendatabot_links_new)
opendatabot_df_new

,eoname,url
0,Відділ освіти Броварської райдержадміністрації,
1,"Управління освіти, молоді та спорту Дубровицьк...",
2,Відділ освіти виконкому Дзержинської районної ...,
3,Відділ освіти Городнянської районної державної...,
4,Управління освіти і науки Чернігівської обласн...,
...,...,...
168,"Управління освіти, культури, туризму, молоді т...",
169,Відділ освіти Гребінківської селищної ради,
170,Відділ освіти Зміївської міської ради Чугуївсь...,
171,"Відділ освіти, культури, молоді та спорту Мар'...",


In [133]:
opendatabot_df_new[opendatabot_df_new.url!='']

,eoname,url
28,"Управління освіти, науки, молоді та спорту Кір...",https://opendatabot.ua/c/02143927
29,Відділ освіти Солонянської райдержадміністраці...,https://opendatabot.ua/c/02142543
30,Відділ освіти Бердянської районної державної а...,https://opendatabot.ua/c/02136169
44,Відділ освіти Броварської районної державної а...,https://opendatabot.ua/c/02143873
45,"Управління освіти, молоді та спорту Володимире...",https://opendatabot.ua/c/02145820
46,"Відділ освіти, молоді та спорту Уманської райо...",https://opendatabot.ua/c/02146959
47,Управління освіти адміністрації Холодногірсько...,https://opendatabot.ua/c/02146305
61,Відділ освіти Кременецької міської ради,https://opendatabot.ua/c/40210966
62,"Відділ освіти, культури, молоді та спорту Корн...",https://opendatabot.ua/c/41090331
63,"Відділ освіти, охорони здоров'я, культури, мол...",https://opendatabot.ua/c/02143146


In [134]:
df = unmatched_link.merge(opendatabot_df_new[['url']], left_on=[unmatched_link['eoname']], right_on=[opendatabot_df_new['eoname']], how='left')

df[~df.url_y.isnull()]

,key_0,eoname,url_x,url_y
62,Відділ освіти Броварської райдержадміністрації,Відділ освіти Броварської райдержадміністрації,NaN,
63,"Управління освіти, молоді та спорту Дубровицьк...","Управління освіти, молоді та спорту Дубровицьк...",NaN,
74,Відділ освіти виконкому Дзержинської районної ...,Відділ освіти виконкому Дзержинської районної ...,NaN,
85,Відділ освіти Городнянської районної державної...,Відділ освіти Городнянської районної державної...,NaN,
86,Управління освіти і науки Чернігівської обласн...,Управління освіти і науки Чернігівської обласн...,NaN,
...,...,...,...,...
3663,"Управління освіти, культури, туризму, молоді т...","Управління освіти, культури, туризму, молоді т...",NaN,
3664,Відділ освіти Гребінківської селищної ради,Відділ освіти Гребінківської селищної ради,NaN,
3665,Відділ освіти Зміївської міської ради Чугуївсь...,Відділ освіти Зміївської міської ради Чугуївсь...,NaN,
3666,"Відділ освіти, культури, молоді та спорту Мар'...","Відділ освіти, культури, молоді та спорту Мар'...",NaN,


In [135]:
unmatched_link = unmatched_link.merge(opendatabot_df_new[['url']], left_on=[unmatched_link['eoname']], right_on=[opendatabot_df_new['eoname']], how='left')
unmatched_link = unmatched_link.drop(columns = ['key_0'], axis=1)
unmatched_link.head()

,eoname,url_x,url_y
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100,NaN
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095,NaN
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345,NaN
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848,NaN
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169,NaN


In [136]:
unmatched_link.loc[unmatched_link['url_x'].isna(), 'url_x'] = ''

In [137]:
unmatched_link.loc[unmatched_link['url_x'] == '', 'url_x'] = unmatched_link.loc[unmatched_link['url_x'] == '', 'url_y']
unmatched_link

,eoname,url_x,url_y
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100,NaN
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095,NaN
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345,NaN
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848,NaN
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169,NaN
...,...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592,NaN
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359,NaN
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142,NaN
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343,NaN


In [138]:
unmatched_link = unmatched_link.drop(columns=['url_y'], axis=1)
unmatched_link = unmatched_link.rename(columns = {'url_x':'url'})
unmatched_link

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343


In [139]:
unmatched_link.drop_duplicates(inplace=True)

In [140]:
unmatched_link.reset_index(drop=True, inplace=True)

In [141]:
unmatched_link[~unmatched_link.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]

,eoname,url
62,Відділ освіти Броварської райдержадміністрації,
63,"Управління освіти, молоді та спорту Дубровицьк...",
74,Відділ освіти виконкому Дзержинської районної ...,
85,Відділ освіти Городнянської районної державної...,
86,Управління освіти і науки Чернігівської обласн...,
...,...,...
3663,"Управління освіти, культури, туризму, молоді т...",
3664,Відділ освіти Гребінківської селищної ради,
3665,Відділ освіти Зміївської міської ради Чугуївсь...,
3666,"Відділ освіти, культури, молоді та спорту Мар'...",


In [142]:
unmatched_link.to_csv('organizations_link.csv',  index=False, encoding='utf-8')